In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes torch
!pip install datasets

In [2]:
!pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 4.2 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

ds = load_dataset("b-mc2/sql-create-context")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [4]:
import pandas as pd

df = pd.DataFrame(ds["train"])

In [5]:
df[:3000].to_csv("data.csv")

In [6]:
df.head()

,answer,question,context
0,SELECT COUNT(*) FROM head WHERE age > 56,How many heads of the departments are older th...,CREATE TABLE head (age INTEGER)
1,"SELECT name, born_state, age FROM head ORDER B...","List the name, born state and age of the heads...","CREATE TABLE head (name VARCHAR, born_state VA..."
2,"SELECT creation, name, budget_in_billions FROM...","List the creation year, name and budget of eac...","CREATE TABLE department (creation VARCHAR, nam..."
3,"SELECT MAX(budget_in_billions), MIN(budget_in_...",What are the maximum and minimum budget of the...,CREATE TABLE department (budget_in_billions IN...
4,SELECT AVG(num_employees) FROM department WHER...,What is the average number of employees of the...,CREATE TABLE department (num_employees INTEGER...


In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-7b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post4: Fast Gemma patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [9]:
alpaca_prompt = """You are expert in converting human queries into SQL.

  ### System Prompt:
  {}

  ### Input:
  {}

  ### Response:
  {}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    system_prompt = examples["context"]
    inputs        = examples["question"]
    outputs       = examples["answer"]
    texts = []
    for instruction, input, output in zip(system_prompt, inputs, outputs):
        text = alpaca_prompt.format(system_prompt, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = load_dataset('csv', data_files='/content/data.csv')
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "finetuned",
    ),
)

trainer_stats = trainer.train()

Map (num_proc=2):   0%|          | 0/3000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 50,003,968


Step,Training Loss
1,0.737400
2,0.737900
3,0.716500
4,0.652100
5,0.653700
6,0.575700
7,0.523200
8,0.475200
9,0.449900
10,0.371500


In [11]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
alpaca_prompt.format(
        "You are an expert in converting English questions to SQL code!. Always use SELECT command. The SQL database has the name 'AI_REPORTS' and has the following columns -Customer_Name,Gross_sales, Net_Sales, Sales_Date, Month, Sale_No, Brand_Name ", # instruction
        "gross sales of January", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 2000, use_cache = True)
tokenizer.batch_decode(outputs)

["<bos>You are expert in converting human queries into SQL.\n\n  ### System Prompt:\n  You are an expert in converting English questions to SQL code!. Always use SELECT command. The SQL database has the name 'AI_REPORTS' and has the following columns -Customer_Name,Gross_sales, Net_Sales, Sales_Date, Month, Sale_No, Brand_Name \n\n  ### Input:\n  gross sales of January\n\n  ### Response:\n  CREATE TABLE AI_REPORTS (Customer_Name VARCHAR, Gross_sales INTEGER, Net_Sales INTEGER, Sales_Date VARCHAR, Month VARCHAR, Sale_No VARCHAR, Brand_Name VARCHAR); CREATE TABLE AI_REPORTS (Customer_Name VARCHAR, Gross_sales INTEGER, Net_Sales INTEGER, Sales_Date VARCHAR, Month VARCHAR, Sale_No VARCHAR, Brand_Name VARCHAR); CREATE TABLE AI_REPORTS (Customer_Name VARCHAR, Gross_sales INTEGER, Net_Sales INTEGER, Sales_Date VARCHAR, Month VARCHAR, Sale_No VARCHAR, Brand_Name VARCHAR); CREATE TABLE AI_REPORTS (Customer_Name VARCHAR, Gross_sales INTEGER, Net_Sales INTEGER, Sales_Date VARCHAR, Month VARCHAR, 